<a href="https://colab.research.google.com/github/Nhorning/Congressional_Survival/blob/master/YG_network_nodup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#install and upgrade thingy to link google sheets and colab notebook. instructions: https://www.pingshiuanchua.com/blog/post/intro-to-colaboratory-and-linking-it-to-google-sheets
!pip install --upgrade -q pygsheets

In [0]:
#authorize link to google sheet
import google.auth
from google.colab import auth

auth.authenticate_user()
import pygsheets
credentials, _ = google.auth.default()
gc = pygsheets.client.Client(credentials)

In [0]:
#open up that google sheet!  replace this \/ with the end of the link to your sheet!  instructions here: https://www.pingshiuanchua.com/blog/post/overpower-your-google-sheets-with-python
sheet = gc.open_by_key('1JnYvTC3a7zm7wufNvmialW2iUgdSpo1c8dxEUHS4FxA')

In [0]:
#Make it a data frame 
worksheet = sheet.worksheet_by_title('Form Responses 1')
df = worksheet.get_as_df()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 16 columns):
Timestamp                                   44 non-null object
Type of Work                                44 non-null object
Details                                     44 non-null object
Name or Psuedonym                           44 non-null object
Organization                                44 non-null object
Website                                     44 non-null object
Phone                                       44 non-null object
The 5-Digit Zipcode Where You Are Active    44 non-null int64
Needs                                       44 non-null object
Thoughts                                    44 non-null object
Yang Gang?                                  44 non-null object
Email                                       36 non-null object
Logo                                        3 non-null object
                                            3 non-null object
Latitude          

In [0]:

#find entries with no e-mail
notblank = [True if x else False for x in df.Email]
#replace those entries with their phone number (let's notfor now)
#df.loc[noemail,'Email'] = df.loc[noemail,'Phone']
#df['Email'] = [str(x)+'@noemail.com' if type(x) == int else x for x in df.Email]

#make all e-mails lowercase to eliminate capitalization issues.
df['Email'] = df.Email.apply(lambda x: x.lower() if x else x)

#find ze duplicate emails - don't flag the latest entry as duplicate.
dups = df.Email.duplicated(keep='last')

#subtract blank entries from the list of dups
dups = dups & notblank

#print the number of dups
print('found', dups.sum(), 'duplicate emails')

#print what is a duplicate
df.loc[dups,:].Email



found 1 duplicate emails


4    fullerjaeson@gmail.com
Name: Email, dtype: object

In [0]:
#remove the duplicate e-mails (filter by everything that is NOT a dup)
nodup_df = df.loc[~dups,:]
nodup_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43 entries, 0 to 43
Data columns (total 16 columns):
Timestamp                                   43 non-null object
Type of Work                                43 non-null object
Details                                     43 non-null object
Name or Psuedonym                           43 non-null object
Organization                                43 non-null object
Website                                     43 non-null object
Phone                                       43 non-null object
The 5-Digit Zipcode Where You Are Active    43 non-null int64
Needs                                       43 non-null object
Thoughts                                    43 non-null object
Yang Gang?                                  43 non-null object
Email                                       35 non-null object
Logo                                        3 non-null object
                                            3 non-null object
Latitude          

In [0]:
#Lets dump it into a second worksheet so we don't risk wiping all the data...

#use the second worksheet
try: 
  second_worksheet = sheet.worksheet_by_title('Form Repsponses No Duplicates')
except:
  #if there isn't one add it and try again
  sheet.add_worksheet('Form Repsponses No Duplicates')
  second_worksheet = sheet.worksheet_by_title('Form Repsponses No Duplicates')

#clear the second worksheet
second_worksheet.clear(start = 'A1')

In [0]:
#dump the non-duplicate sheet in - Ta Da!
second_worksheet.set_dataframe(nodup_df, start = 'A1', nan = '')